### First Example: Impressions by Key/Value
- Will use HI as an example but the same thing needs to be done for all the other keys/values

In [1]:
import pandas as pd
import numpy as np
from google.cloud import bigquery
from sklearn.preprocessing import RobustScaler
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:,.2f}'.format

bq_client = bigquery.Client()

sql_hi = """
   SELECT
      *
    FROM
      `api-project-901373404215.fpd_imp.hi`
    """

df_hi = bq_client.query(sql_hi).to_dataframe()
df_hi['date'] = pd.to_datetime(df_hi['date'])
df_hi['hi'] = df_hi['hi'].astype(str)

print(df_hi.shape)
df_hi.head()

(349758, 5)


,date,hi,dfp_zone,device_category,hi_imps
0,2021-03-17,6,forbes:home,Tablet,35
1,2021-03-17,5,forbes:home,Tablet,257
2,2021-03-17,1,forbes:home,Tablet,58
3,2021-03-17,3,forbes:home,Tablet,469
4,2021-03-17,4,forbes:home,Tablet,453


In [2]:
# Impressions by hhi category
hi_summ = df_hi.groupby('hi').sum().reset_index()
hi_summ.sort_values(by='hi')

,hi,hi_imps
0,1,24433865
1,2,579768378
2,3,995206349
3,4,606424081
4,5,906908467
5,6,68443747
6,7,11852441
7,8,1796693


In [3]:
# Map the numbers to labels
hi_mapping = {
    '1': 'under_25k',
    '2': '_25_49999',
    '3': '_50_74999',
    '4': '_75_99999',
    '5': '_100_149999',
    '6': '_150_199999',
    '7': '_200_249999',
    '8': '_250_499999',
    '9': '_500_999999',
    '10': '_1M_plus'
}

df_hi['hi'] = df_hi['hi'].map(hi_mapping)

df_hi.head()

,date,hi,dfp_zone,device_category,hi_imps
0,2021-03-17,_150_199999,forbes:home,Tablet,35
1,2021-03-17,_100_149999,forbes:home,Tablet,257
2,2021-03-17,under_25k,forbes:home,Tablet,58
3,2021-03-17,_50_74999,forbes:home,Tablet,469
4,2021-03-17,_75_99999,forbes:home,Tablet,453


In [4]:
# Impressions by hhi category label
hi_summ = df_hi.groupby('hi').sum().reset_index()
hi_summ.sort_values(by='hi_imps', ascending=False)

,hi,hi_imps
5,_50_74999,995206349
0,_100_149999,906908467
6,_75_99999,606424081
4,_25_49999,579768378
1,_150_199999,68443747
7,under_25k,24433865
2,_200_249999,11852441
3,_250_499999,1796693


In [5]:
# Map to the correct device category
device_dict = {
    'Desktop': 'desktop',
    'Tablet': 'desktop',
    'Smartphone': 'mobile'
}

df_hi['device_category'] = df_hi['device_category'].map(device_dict)

# Remove all the other device categories
df_hi = df_hi.loc[df_hi['device_category'].isin(['desktop', 'mobile'])]

df_hi

,date,hi,dfp_zone,device_category,hi_imps
0,2021-03-17,_150_199999,forbes:home,desktop,35
1,2021-03-17,_100_149999,forbes:home,desktop,257
2,2021-03-17,under_25k,forbes:home,desktop,58
3,2021-03-17,_50_74999,forbes:home,desktop,469
4,2021-03-17,_75_99999,forbes:home,desktop,453
...,...,...,...,...,...
349706,2021-03-16,_50_74999,fdc.forbes:article-delta-b-standard-default-st...,mobile,1
349707,2021-03-16,_100_149999,fdcmobile:article-amp-topline-nonsubscriber-st...,mobile,19
349708,2021-03-16,_100_149999,fdcmobile:article-amp-masthead-nonsubscriber-s...,mobile,3
349709,2021-03-16,_100_149999,fdcmobile:article-amp-standard-nonsubscriber-s...,mobile,243


In [6]:
len(df_hi['dfp_zone'].unique())

606

In [7]:
# Top 20 zones
df_zone = df_hi.groupby('dfp_zone').sum().reset_index()

df_zone.sort_values(
    by='hi_imps',
    ascending=False
).head(20)

,dfp_zone,hi_imps
311,fdc.forbes:article-standard-default-standard,1482792120
549,fdcmobile:article-standard-default-standard,377320617
356,fdcmobile:article-amp-standard-default-standard,334589868
325,fdc.forbes:article-topline-default-standard,176326571
267,fdc.forbes:article-finds-default-standard,95689794
557,fdcmobile:article-topline-default-standard,82181818
103,fdc.forbes:article-delta-b-standard-default-st...,58748046
242,fdc.forbes:article-delta-g-standard-default-st...,57813484
359,fdcmobile:article-amp-topline-default-standard,55927019
542,fdcmobile:article-premium-default-standard,54499783


In [8]:
# Amp zones only
df_hi['dfp_zone'] = df_hi['dfp_zone'].fillna('none')
df_amp = df_hi.loc[df_hi['dfp_zone'].str.contains(':article-amp-', regex=False)]
df_amp = df_amp.groupby('dfp_zone').sum().reset_index()
df_amp.sort_values(by='hi_imps', ascending=False)

,dfp_zone,hi_imps
16,fdcmobile:article-amp-standard-default-standard,334589868
19,fdcmobile:article-amp-topline-default-standard,55927019
1,fdcmobile:article-amp-finds-default-standard,22125165
10,fdcmobile:article-amp-masthead-default-standard,10338006
13,fdcmobile:article-amp-premium-default-standard,4565510
18,fdcmobile:article-amp-standard-subscriber-stan...,1385123
21,fdcmobile:article-amp-topline-subscriber-standard,718443
6,fdcmobile:article-amp-live-default-standard,186243
12,fdcmobile:article-amp-masthead-subscriber-stan...,69875
15,fdcmobile:article-amp-premium-subscriber-standard,32811


In [9]:
# Map the dfp zone to amp vs non-amp
df_hi['dfp_zone'] = np.where(
    df_hi['dfp_zone'].str.contains(':article-amp-', regex=False),
    'amp',
    None
)

df_hi['dfp_zone'] = df_hi['dfp_zone'].fillna(df_hi['device_category'])

df_hi

,date,hi,dfp_zone,device_category,hi_imps
0,2021-03-17,_150_199999,desktop,desktop,35
1,2021-03-17,_100_149999,desktop,desktop,257
2,2021-03-17,under_25k,desktop,desktop,58
3,2021-03-17,_50_74999,desktop,desktop,469
4,2021-03-17,_75_99999,desktop,desktop,453
...,...,...,...,...,...
349706,2021-03-16,_50_74999,mobile,mobile,1
349707,2021-03-16,_100_149999,amp,mobile,19
349708,2021-03-16,_100_149999,amp,mobile,3
349709,2021-03-16,_100_149999,amp,mobile,243


In [10]:
df_hi.groupby('dfp_zone').sum()

,hi_imps
dfp_zone,
amp,429976394
desktop,2105488127
mobile,658147862


In [11]:
# TOTAL

# Group by to get the total daily hi_imps
df_total_hi = df_hi.groupby('date').sum().reset_index()
prefix = 'total_'
df_total_hi.columns = [prefix + col if col != 'date' else col for col in df_total_hi.columns]
df_total_hi

,date,total_hi_imps
0,2020-08-28,10779213
1,2020-08-29,12315604
2,2020-08-30,13676909
3,2020-08-31,20539971
4,2020-09-01,20560628
...,...,...
243,2021-04-28,15105377
244,2021-04-29,13167114
245,2021-04-30,12062505
246,2021-05-01,10339050


In [12]:
# Total by hi values
df_total_hi_val = df_hi.groupby(['date', 'hi']).sum().reset_index()

df_total_hi_val = pd.pivot_table(
    df_total_hi_val,
    index='date',
    columns='hi',
    values='hi_imps',
    aggfunc='sum'
).reset_index()

# Add a prefix for easy filtering later
prefix = 'total_'
df_total_hi_val.columns = [prefix + col if col != 'date' else col for col in df_total_hi_val.columns]

df_total_hi_val

,date,total__100_149999,total__150_199999,total__200_249999,total__250_499999,total__25_49999,total__50_74999,total__75_99999,total_under_25k
0,2020-08-28,1960809,270173,43593,5321,2284860,3778018,2337903,98536
1,2020-08-29,2183349,315085,48279,6015,2609386,4367202,2695151,91137
2,2020-08-30,2377936,327596,51166,6187,2952899,4892475,2957549,111101
3,2020-08-31,3797974,510013,80186,10242,4339951,7129440,4474405,197760
4,2020-09-01,3759071,522925,79899,11534,4396446,7153313,4443360,194080
...,...,...,...,...,...,...,...,...,...
243,2021-04-28,2170655,319390,52732,9436,2475063,7444154,2523710,110237
244,2021-04-29,2046115,289272,50800,8569,2695109,5603598,2372626,101025
245,2021-04-30,1844650,264253,45128,6962,2473092,5211853,2127073,89494
246,2021-05-01,1391949,193169,31164,5184,1743023,5150154,1765018,59389


In [13]:
# Probably don't need device by hi or device by hi values

In [14]:
# Merge the dfs together so each row is a single date
df_hi = pd.merge(
    df_total_hi,
    df_total_hi_val,
    on='date',
    how='left'
)

df_hi

,date,total_hi_imps,total__100_149999,total__150_199999,total__200_249999,total__250_499999,total__25_49999,total__50_74999,total__75_99999,total_under_25k
0,2020-08-28,10779213,1960809,270173,43593,5321,2284860,3778018,2337903,98536
1,2020-08-29,12315604,2183349,315085,48279,6015,2609386,4367202,2695151,91137
2,2020-08-30,13676909,2377936,327596,51166,6187,2952899,4892475,2957549,111101
3,2020-08-31,20539971,3797974,510013,80186,10242,4339951,7129440,4474405,197760
4,2020-09-01,20560628,3759071,522925,79899,11534,4396446,7153313,4443360,194080
...,...,...,...,...,...,...,...,...,...,...
243,2021-04-28,15105377,2170655,319390,52732,9436,2475063,7444154,2523710,110237
244,2021-04-29,13167114,2046115,289272,50800,8569,2695109,5603598,2372626,101025
245,2021-04-30,12062505,1844650,264253,45128,6962,2473092,5211853,2127073,89494
246,2021-05-01,10339050,1391949,193169,31164,5184,1743023,5150154,1765018,59389


In [15]:
# You also need to do the totals (overall and by device/AMP)
# There's no total table so you have to add all the sub tables together

In [16]:
sql_total = """
    WITH hi AS (
        SELECT
            date,
            dfp_zone,
            device_category,
            SUM(hi_imps) AS hi_imps
        FROM
        `api-project-901373404215.fpd_imp.hi`
        GROUP BY 
            date,
            dfp_zone,
            device_category
    ),

    jt AS (
        SELECT
            date,
            dfp_zone,
            device_category,
            SUM(jt_imps) AS jt_imps
        FROM
        `api-project-901373404215.fpd_imp.jt`
        GROUP BY 
            date,
            dfp_zone,
            device_category
    ),

    fuse AS (
        SELECT
            date,
            dfp_zone,
            device_category,
            SUM(fuse_imps) AS fuse_imps
        FROM
        `api-project-901373404215.fpd_imp.fuse`
        GROUP BY 
            date,
            dfp_zone,
            device_category
    ),

    fuse2 AS (
        SELECT
            date,
            dfp_zone,
            device_category,
            SUM(fuse2_imps) AS fuse2_imps
        FROM
        `api-project-901373404215.fpd_imp.fuse2`
        GROUP BY 
            date,
            dfp_zone,
            device_category
    ),

    fbs_aa AS (
        SELECT
            date,
            dfp_zone,
            device_category,
            SUM(fbs_aa_imps) AS fbs_aa_imps
        FROM
        `api-project-901373404215.fpd_imp.fbs_aa`
        GROUP BY 
            date,
            dfp_zone,
            device_category
    ),

    fbs_dm AS (
        SELECT
            date,
            dfp_zone,
            device_category,
            SUM(dm_imps) AS fbs_dm_imps
        FROM
            `api-project-901373404215.fpd_imp.fbs_dm`
        GROUP BY 
            date,
            dfp_zone,
            device_category
    )

    SELECT 
        hi.date,
        hi.dfp_zone,
        hi.device_category,
        hi_imps,
        jt_imps,
        fuse_imps,
        fuse2_imps,
        fbs_aa_imps,
        fbs_dm_imps
    FROM
        hi 
    LEFT OUTER JOIN 
        jt 
    ON 
        hi.date = jt.date
        AND hi.dfp_zone = jt.dfp_zone
        AND hi.device_category = jt.device_category
    LEFT OUTER JOIN 
        fuse 
    ON 
        hi.date = fuse.date
        AND hi.dfp_zone = fuse.dfp_zone
        AND hi.device_category = fuse.device_category
    LEFT OUTER JOIN 
        fuse2
    ON 
        hi.date = fuse2.date
        AND hi.dfp_zone = fuse2.dfp_zone
        AND hi.device_category = fuse2.device_category
    LEFT OUTER JOIN 
        fbs_aa
    ON 
        hi.date = fbs_aa.date
        AND hi.dfp_zone = fbs_aa.dfp_zone
        AND hi.device_category = fbs_aa.device_category
    LEFT OUTER JOIN 
        fbs_dm
    ON 
        hi.date = fbs_dm.date
        AND hi.dfp_zone = fbs_dm.dfp_zone
        AND hi.device_category = fbs_dm.device_category
    """

df_total = bq_client.query(sql_total).to_dataframe()
df_total['date'] = pd.to_datetime(df_total['date'])

df_total = df_total.fillna(0)

print(df_total.shape)
df_total.head()

(83833, 9)


,date,dfp_zone,device_category,hi_imps,jt_imps,fuse_imps,fuse2_imps,fbs_aa_imps,fbs_dm_imps
0,2020-11-19,fdc.forbes:article-delta-g-masthead-default-st...,Tablet,15,0.00,0.00,0.00,0.00,0.00
1,2020-11-19,forbes:series,Desktop,88,12.00,20.00,63.00,0.00,0.00
2,2020-11-19,fdc.forbes:article-delta-b-topline,Desktop,130,11.00,55.00,220.00,0.00,0.00
3,2020-11-19,aax.forbes:fdc.forbes-article-blue-standard,Desktop,1569,240.00,248.00,460.00,0.00,3.00
4,2020-11-19,fdc.forbes:article-premium-default-standard,Desktop,79135,"15,674.00","14,631.00","33,826.00",0.00,"3,616.00"


In [17]:
df_total.groupby('date').sum()

,hi_imps,jt_imps,fuse_imps,fuse2_imps,fbs_aa_imps,fbs_dm_imps
date,,,,,,
2020-08-28,10786665,"2,484,825.00","19,590,463.00",0.00,0.00,0.00
2020-08-29,12326048,"2,881,412.00","15,043,525.00",0.00,0.00,0.00
2020-08-30,13687395,"3,521,062.00","18,204,835.00",0.00,0.00,0.00
2020-08-31,20549120,"5,350,304.00","28,286,147.00",0.00,0.00,0.00
2020-09-01,20569578,"5,056,706.00","26,789,794.00",0.00,0.00,0.00
...,...,...,...,...,...,...
2021-04-28,15108513,"301,392.00","4,828,358.00","17,179,356.00","687,555.00","628,916.00"
2021-04-29,13169286,"275,902.00","4,323,535.00","15,326,569.00","603,678.00","574,237.00"
2021-04-30,12065370,"244,765.00","3,746,938.00","13,210,690.00","513,190.00","497,243.00"


In [18]:
# Its a little higher bc we are excluding some device categories which are super small
df_total_hi

,date,total_hi_imps
0,2020-08-28,10779213
1,2020-08-29,12315604
2,2020-08-30,13676909
3,2020-08-31,20539971
4,2020-09-01,20560628
...,...,...
243,2021-04-28,15105377
244,2021-04-29,13167114
245,2021-04-30,12062505
246,2021-05-01,10339050
